In [6]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout


In [7]:
df = pd.read_csv("E:\citysense360\data\pollution\city_day.csv")
df.head()


,City,Date,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket
0,Ahmedabad,2015-01-01,NaN,NaN,0.92,18.22,17.15,NaN,0.92,27.64,133.36,0.00,0.02,0.00,NaN,NaN
1,Ahmedabad,2015-01-02,NaN,NaN,0.97,15.69,16.46,NaN,0.97,24.55,34.06,3.68,5.50,3.77,NaN,NaN
2,Ahmedabad,2015-01-03,NaN,NaN,17.40,19.30,29.70,NaN,17.40,29.07,30.70,6.80,16.40,2.25,NaN,NaN
3,Ahmedabad,2015-01-04,NaN,NaN,1.70,18.48,17.97,NaN,1.70,18.59,36.08,4.43,10.14,1.00,NaN,NaN
4,Ahmedabad,2015-01-05,NaN,NaN,22.10,21.42,37.76,NaN,22.10,39.33,39.31,7.01,18.89,2.78,NaN,NaN


In [8]:
features = [
    "PM2.5", "PM10", "NO2", "SO2", "CO", "O3"
]

df = df.dropna(subset=features + ["AQI"])


In [9]:
X = df[features]
y = df["AQI"]


In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [11]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [12]:
model = Sequential([
    Dense(64, activation="relu", input_shape=(X_train.shape[1],)),
    Dropout(0.2),
    Dense(32, activation="relu"),
    Dense(1)  # AQI value
])


In [13]:
model.compile(
    optimizer="adam",
    loss="mse",
    metrics=["mae"]
)


In [14]:
history = model.fit(
    X_train, y_train,
    validation_split=0.1,
    epochs=30,
    batch_size=32,
    verbose=1
)


Epoch 1/30


361/361 [==============================] - 2s 2ms/step - loss: 20466.0156 - mae: 101.9252 - val_loss: 6170.7207 - val_mae: 56.7796
Epoch 2/30
361/361 [==============================] - 1s 2ms/step - loss: 5595.8335 - mae: 54.8629 - val_loss: 3988.7280 - val_mae: 46.6722
Epoch 3/30
361/361 [==============================] - 1s 2ms/step - loss: 3436.5767 - mae: 40.5728 - val_loss: 2275.5823 - val_mae: 30.1471
Epoch 4/30
361/361 [==============================] - 1s 2ms/step - loss: 2269.1062 - mae: 29.9437 - val_loss: 1828.8380 - val_mae: 27.1835
Epoch 5/30
361/361 [==============================] - 1s 4ms/step - loss: 1999.1512 - mae: 27.9574 - val_loss: 1691.2666 - val_mae: 25.8941
Epoch 6/30
361/361 [==============================] - 1s 3ms/step - loss: 1893.7662 - mae: 26.9475 - val_loss: 1578.3148 - val_mae: 24.7494
Epoch 7/30
361/361 [==============================] - 1s 2ms/step - loss: 1767.0051 - mae: 26.4072 - val_loss: 1479.3887 - val_mae: 24.1389
Epoch 8/30
361/3

In [15]:
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)

print("MAE:", mae)


101/101 [==============================] - 0s 1ms/step
MAE: 20.114792184037466


In [16]:
model.save("E:/citysense360/models/aqi_regression_model.h5")
print("AQI model saved")


AQI model saved


D:\anaconda\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [17]:
def aqi_category(aqi):
    if aqi <= 50:
        return "Good"
    elif aqi <= 100:
        return "Moderate"
    elif aqi <= 200:
        return "Poor"
    elif aqi <= 300:
        return "Very Poor"
    else:
        return "Severe"


In [18]:
import numpy as np

sample_input = {
    "PM2.5": 85,
    "PM10": 140,
    "NO2": 35,
    "SO2": 10,
    "CO": 1.2,
    "O3": 28
}


In [19]:
# Convert to array in correct order
X_new = np.array([[ 
    sample_input["PM2.5"],
    sample_input["PM10"],
    sample_input["NO2"],
    sample_input["SO2"],
    sample_input["CO"],
    sample_input["O3"]
]])

# Scale input
X_new_scaled = scaler.transform(X_new)

# Predict AQI
predicted_aqi = model.predict(X_new_scaled)[0][0]

print("Predicted AQI:", round(predicted_aqi, 2))
print("AQI Category:", aqi_category(predicted_aqi))


1/1 [==============================] - 0s 21ms/step
Predicted AQI: 167.54
AQI Category: Poor


D:\anaconda\anaconda3\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [22]:
import joblib
joblib.dump(scaler, "E:/citysense360/models/aqi_scaler.pkl")
print("Scaler saved successfully")


Scaler saved successfully
